In [24]:
import os
import pandas as pd
from BaselineRemoval import BaselineRemoval
from scipy import signal
import pickle

def find_peaks(y, cwt=False, smooth=False, baseline=False):
    window, deg = 15, 2     
    smoothed = y
    if smooth:
        smoothed = signal.savgol_filter(y, window, deg, deriv=0)

    z = smoothed
    if baseline:
        brm = BaselineRemoval(smoothed)
        z = brm.ZhangFit(lambda_=400,repitition=15, porder=1)
    if cwt == True:
        peaks = signal.find_peaks_cwt(z, [20])  
    else:
        dist, prom = 100, 600
        peaks,_ = signal.find_peaks(z, distance=dist, prominence=prom)
    return peaks, z

def similar_to(pk1, pk2, method='jaccard', rank=5):
    if method == 'rank':
        return sim.rank_similarity(pk1, pk2, good_with=rank)
    else:
        return sim.jaccard_similarity(pk1, pk2)
    

DATA_DIR_PATH = "datasets/export"
PEAK_DIR_PATH = "peaks"
files =[
    "L185_0_A9_1", "L185_0_A10_1",
    "L186_0_A11_1", "L186_0_A12_1",
    "L187_0_B1_1", "L187_0_B2_1",
    "L291_0_A1_1", "L291_0_A2_1",
    "L100_0_G7_1", "L100_0_G8_1",
    "L101_0_A1_1", "L101_0_A2_1",
    "L103_0_A5_1", "L103_0_A6_1",
    "L125_0_E1_1", "L125_0_E2_1",
    "L126_0_E3_1", "L126_0_E4_1",
    "L128_0_E7_1", "L128_0_E8_1",   
]
peaks_extracted = []
n = len(files)
for i in range(n):
    df = None
    peak_file = open(f'{PEAK_DIR_PATH}/{files[i]}_peaks.txt', 'wb')
    df = pd.read_table(f"{DATA_DIR_PATH}/{files[i]}.txt",sep=" ", header=None,names=['m/z', 'intensity']) 
    x, y = df['m/z'].to_numpy(), df['intensity'].to_numpy()
    %time peaks, z =  find_peaks(y, cwt=True, smooth=False, baseline=False)
    print(f'{files[i]} peaks: {len(peaks)}')
    pickle.dump(peaks, peak_file)
    peaks_extracted += [(x[peaks], z[peaks])]
    peak_file.close()
    

CPU times: user 9.78 s, sys: 7.05 ms, total: 9.79 s
Wall time: 9.79 s
L185_0_A9_1 peaks: 195
CPU times: user 9.92 s, sys: 9.99 ms, total: 9.93 s
Wall time: 9.94 s
L185_0_A10_1 peaks: 202
CPU times: user 9.7 s, sys: 3.39 ms, total: 9.71 s
Wall time: 9.71 s
L186_0_A11_1 peaks: 213
CPU times: user 9.69 s, sys: 3.89 ms, total: 9.69 s
Wall time: 9.69 s
L186_0_A12_1 peaks: 219
CPU times: user 9.73 s, sys: 3.28 ms, total: 9.73 s
Wall time: 9.73 s
L187_0_B1_1 peaks: 211
CPU times: user 9.76 s, sys: 4.8 ms, total: 9.77 s
Wall time: 9.77 s
L187_0_B2_1 peaks: 215
CPU times: user 9.81 s, sys: 4.1 ms, total: 9.81 s
Wall time: 9.81 s
L291_0_A1_1 peaks: 215
CPU times: user 9.75 s, sys: 5.34 ms, total: 9.76 s
Wall time: 9.76 s
L291_0_A2_1 peaks: 201
CPU times: user 9.73 s, sys: 4.28 ms, total: 9.74 s
Wall time: 9.74 s
L100_0_G7_1 peaks: 207
CPU times: user 9.68 s, sys: 3.68 ms, total: 9.68 s
Wall time: 9.68 s
L100_0_G8_1 peaks: 224
CPU times: user 10.1 s, sys: 16.7 ms, total: 10.1 s
Wall time: 10.1 s


In [25]:
import importlib

In [31]:
import similarity as sim
importlib.reload(sim)
rs = []
n = len(files)
for i in range(n-1):
    for j in range(i+1,n):
        if i == j:
            continue

        score = similar_to(peaks_extracted[i], peaks_extracted[j], method='rank', rank=2)
        rs += [(i, j, score)]

rs = sorted(rs, key=lambda x: x[2], reverse=True)
for k, v in enumerate(rs):
    i, j, s = v
    tag = '*' if files[i][:4]==files[j][:4] else '' # tag the same strain id
    print(f'#{k+1:2d}{tag}:\t{files[i]} & {files[j]}\t{s:.4f}{tag}')

# 1*:	L128_0_E7_1 & L128_0_E8_1	0.1397*
# 2:	L185_0_A10_1 & L186_0_A11_1	0.1212
# 3:	L125_0_E2_1 & L128_0_E7_1	0.1014
# 4:	L125_0_E2_1 & L128_0_E8_1	0.1011
# 5:	L103_0_A5_1 & L128_0_E8_1	0.0854
# 6*:	L100_0_G7_1 & L100_0_G8_1	0.0810*
# 7*:	L185_0_A9_1 & L185_0_A10_1	0.0791*
# 8:	L103_0_A6_1 & L125_0_E1_1	0.0777
# 9:	L185_0_A10_1 & L186_0_A12_1	0.0728
#10:	L185_0_A9_1 & L187_0_B2_1	0.0726
#11*:	L186_0_A11_1 & L186_0_A12_1	0.0696*
#12:	L101_0_A2_1 & L125_0_E2_1	0.0682
#13:	L101_0_A1_1 & L125_0_E1_1	0.0677
#14:	L103_0_A6_1 & L126_0_E4_1	0.0676
#15:	L186_0_A11_1 & L187_0_B2_1	0.0667
#16:	L185_0_A9_1 & L186_0_A12_1	0.0662
#17:	L101_0_A1_1 & L103_0_A6_1	0.0629
#18:	L185_0_A10_1 & L187_0_B2_1	0.0627
#19:	L101_0_A2_1 & L128_0_E7_1	0.0618
#20:	L185_0_A10_1 & L126_0_E3_1	0.0610
#21:	L103_0_A6_1 & L128_0_E7_1	0.0609
#22:	L187_0_B2_1 & L291_0_A2_1	0.0605
#23:	L101_0_A2_1 & L128_0_E8_1	0.0604
#24:	L185_0_A9_1 & L186_0_A11_1	0.0602
#25:	L185_0_A10_1 & L103_0_A6_1	0.0594
#26*:	L125_0_E1_1 & L125_0_E2